# Book insights: Estrategia de Contenido Post-Pandemia.

## El Problema: El mercado de lectura digital está saturado tras el auge de la lectura debido al confinamiento que causó la pandemia. Las startups de contenido de lectura en digital necesitan saber qué tipo de autores, editoriales y libros generan mayor interés y compromiso para diseñar una propuesta de valor.



## Objetivos:
* ### Auditoría de Volumen: Cuantificar la oferta de contenido moderno.

* ### Calidad y Relevancia: Identificar quiénes son los mas relevantes de la plataforma; editoriales y autores, filtrando el contenido de baja calidad o folletos menores a 50 páginas.

* ### Análisis de Usuario: Entender el comportamiento de los usuarios más activos para identificar patrones de reseñas.

### Conexión con la Base de Datos


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
db_config = {
 'user': 'practicum_student', # username
 'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
 'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
 'port': 5432, # connection port
 'db': 'data-analyst-final-project-db' # the name of the database
 }
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})



: 

### Exploración Preliminar de Tablas.

In [ ]:
# Definimos una lista con los nombres de nuestras tablas para iterar por cada una y revisar.
tablas =['books', 'authors', 'publishers', 'ratings', 'reviews']

for tabla in tablas:
  print(f'\n Inspeccionando el contenido de la tabla {tabla}\n')
  query = f'SELECT * FROM {tabla} LIMIT 5'
# Ejecutamos la consulta con Pandas y el engine que creamos
  df_temp = pd.io.sql.read_sql(query, con=engine)

  display(df_temp)
  print("-" * 30)


### Observaciones:
#### Estas tablas contienen todos los elementos necesarios para abordar nuestros tres objetivos principales: books para volumen y filtrar calidad, authors y publishers para identificar relevancia, y ratings junto con reviews para el análisis del comportamiento del usuario y su engagement.

## 1) Encontrar el número de libros publicados después del 1 de enero de 2000.

In [ ]:
query_1 = """
SELECT
    COUNT (book_id) AS total_libros_modernos
FROM
    books
WHERE
    publication_date > '2000-01-01'
"""

resultado_conteo = pd.io.sql.read_sql(query_1, con=engine)
print('El Total de Libros Publicados después del año 2000 es: \n')
display(resultado_conteo)

## 2) Encontrar el número de reseñas de usuarios y la calificación promedio para cada libro.

In [ ]:
query_2 = """
SELECT
    b.title AS titulo_libro,
    COUNT(DISTINCT rev.review_id) AS total_reseñas,
    ROUND(AVG(rat.rating), 2) AS rating_promedio
FROM
    books AS b
LEFT JOIN
    reviews AS rev ON b.book_id = rev.book_id
LEFT JOIN
    ratings AS rat ON b.book_id = rat.book_id
GROUP BY
    b.book_id, b.title
ORDER BY
    rating_promedio DESC;
"""

resultado_2 = pd.io.sql.read_sql(query_2, con=engine)
print("Análisis de Calidad por Libro en base a Ratings y Reseñas\n")
display(resultado_2.head(10)) # Mostramos los 10 mejores

## 3) Identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas

In [ ]:
query_3 = """
SELECT
    p.publisher AS editorial,
    COUNT(b.book_id) AS cantidad_libros
FROM
    publishers AS p
INNER JOIN
    books AS b ON p.publisher_id = b.publisher_id
WHERE
    b.num_pages > 50
GROUP BY
    p.publisher
ORDER BY
    cantidad_libros DESC
LIMIT 1;
"""

resultado_3 = pd.io.sql.read_sql(query_3, con=engine)
print("La Editorial con mayor producción de contenido en 'libros' es:")
display(resultado_3)

## 4) Identificar al autor que tiene la más alta calificación promedio del libro: miraremos solo los libros con al menos 50 calificaciones.

In [ ]:
query_4 = """
SELECT
    a.author AS autor,
    ROUND(AVG(r.rating), 4) AS calificacion_promedio
FROM
    authors AS a
JOIN
    books AS b ON a.author_id = b.author_id
JOIN
    ratings AS r ON b.book_id = r.book_id
WHERE
    b.book_id IN (
        -- INICIO DE SUBCONSULTA (La Lista VIP)
        SELECT book_id
        FROM ratings
        GROUP BY book_id
        HAVING COUNT(rating_id) >= 50
        -- FIN DE SUBCONSULTA
    )
GROUP BY
    a.author
ORDER BY
    calificacion_promedio DESC
LIMIT 1;
"""

resultado_4 = pd.io.sql.read_sql(query_4, con=engine)
print("El Autor con mejor crítica en libros populares es:")
display(resultado_4)

## 5) Encontrar el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [ ]:
query_5_cte = """
WITH usuarios_top AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 50
),
conteo_reseñas AS (
    SELECT
        r.username,
        COUNT(r.review_id) as n_reseñas
    FROM reviews r
    WHERE r.username IN (SELECT username FROM usuarios_top)
    GROUP BY r.username
)
SELECT AVG(n_reseñas) as promedio_final
FROM conteo_reseñas;
"""
# Ejecución
df_final = pd.io.sql.read_sql(query_5_cte, con=engine)
display(df_final)

## Gráficos y visualizaciones


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Consulta para el Top 10
query_top_autores = """
SELECT
    a.author,
    AVG(r.rating) as avg_rating
FROM authors a
JOIN books b ON a.author_id = b.author_id
JOIN ratings r ON b.book_id = r.book_id
WHERE b.book_id IN (
    SELECT book_id FROM ratings GROUP BY book_id HAVING COUNT(rating_id) >= 50
)
GROUP BY a.author
ORDER BY avg_rating DESC
LIMIT 10;
"""

df_top_autores = pd.io.sql.read_sql(query_top_autores, con=engine)

# Creamos del gráfico
plt.figure(figsize=(12, 6))
sns.barplot(x='avg_rating', y='author', data=df_top_autores, palette='viridis', hue='author',legend=False)

plt.title('Top 10 Autores por Calificación Promedio. Libros con más de 50 Ratings', fontsize=16)
plt.xlabel('Calificación Promedio', fontsize=14)
plt.ylabel('Autor', fontsize=14)
plt.xlim(3.5, 5)
plt.grid(axis='x', linestyle='--', alpha=0.7)

plt.show()

## Hallazgos
### Capacidad Editorial
#### Editorial Líder: Penguin Books se posiciona como el principal proveedor con 42 títulos que cumplen con los estándares de calidad; mas de 50 páginas.

### Propuesta de Valor:
####  Se recomienda fortalecer la alianza con este proveedor para garantizar contenido de largo formato.

### Ranking de Autores
#### El autor de culto: J.K. Rowling/Mary GrandPré lidera con la calificación promedio más alta en el segmento de libros con alta atracción mas de 50 votos.

### Diversificación:
#### Autores como Markus Zusak y J.R.R. Tolkien muestran una retención de calidad similar, lo que permite diversificar las recomendaciones premium sin perder prestigio.

### Comportamiento del "usuario activo":
#### Los usuarios que han calificado más de 50 libros que es nuestra base más leal escriben un promedio de 24.33 reseñas de texto.

### Insights:
#### Estos usuarios no solo consumen, sino que actúan como filtros o mejoradores de contenido. Fomentar su actividad es importante para generar confianza en nuevos lectores.

## Conclusiones y Recomendaciones
* ### Enfoque en Libros Modernos: Como el análisis se centró en publicaciones después del año 2000, la estrategia de marketing debería apuntar a un público joven-adulto que este interesado en la literatura contemporánea.

* ### Gamificación para Reseñas: El "Power User" ya es activo; crear un sistema de insignias motivacionales para quienes superan las 20 reseñas podría aumentar la densidad de críticas en libros nuevos e incentivarlos a poner sus reseñas.

* ### Filtro de Calidad Permanente: Se recomienda implementar en la App un filtro de búsqueda por "Relevancia" para evitar que libros con una sola calificación distorsionen los rankings de los usuarios.